In [ ]:
import numpy as np

from bofire.data_models.domain.api import Domain
from bofire.data_models.features.api import ContinuousInput, ContinuousOutput, TaskInput
from bofire.data_models.strategies.predictives.multi_fidelity import (
    MultiFidelityStrategy as DataModel,
)
from bofire.data_models.surrogates.botorch_surrogates import (
    BotorchSurrogates,
    MultiTaskGPSurrogate,
)
from bofire.strategies.predictives.multi_fidelity import MultiFidelityStrategy

In [ ]:
task_input = TaskInput(
    key="task",
    fidelities=[0, 1],
    categories=["task_A", "task_B"],
    allowed=[True, False],
)

domain = Domain.from_lists(
    inputs=[ContinuousInput(key="x", bounds=(0, 1)), task_input],
    outputs=[ContinuousOutput(key="y")],
)

In [ ]:
data_model = DataModel(
    domain=domain,
    surrogate_specs=BotorchSurrogates(
        surrogates=[
            MultiTaskGPSurrogate(
                inputs=domain.inputs,
                outputs=domain.outputs,
            )
        ]
    ),
)

strategy = MultiFidelityStrategy(data_model=data_model)

c:\Users\tobyb\phd\bofire\venv\lib\site-packages\pydantic\main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


In [ ]:
def f(X):
    x = X["x"]
    task_A = np.sin(x * 2 * np.pi)
    task_B = 0.9 * np.sin(x * 2 * np.pi) - 0.2 + 0.2 * np.cos(x * 3 * np.pi)
    return np.where(X["task"] == "task_A", task_A, task_B)


N = 4
seed = 1
np.random.seed(seed)
experiments = domain.inputs.sample(N, seed=seed)
experiments["task"] = np.random.choice(task_input.categories, N)
experiments["y"] = f(experiments)
print(experiments)

          x    task         y
0  0.322903  task_B  0.408186
1  0.323703  task_B  0.406032
2  0.222289  task_A  0.984881
3  0.127538  task_A  0.718295


In [ ]:
strategy.tell(experiments)

for _ in range(20):
    next_experiment = strategy.ask(1)
    next_experiment["y"] = f(next_experiment)
    strategy.tell(next_experiment)

c:\Users\tobyb\phd\bofire\venv\lib\site-packages\linear_operator\utils\interpolation.py:71: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:653.)
  summing_matrix = cls(summing_matrix_indices, summing_matrix_values, size)
c:\Users\tobyb\phd\bofire\venv\lib\site-packages\botorch\acquisition\cached_cholesky.py:87: RuntimeWarning: `cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.multitask.MultiTaskGP'>. Setting `cache_root = False`.
  warnings.warn(
c:\Users\tobyb\phd\bofire\venv\lib\site-packages\botorch\acquisition\cached_cholesky.py:87: RuntimeWarning: `cache_root` is only supported for GPyTorchModels that are not MultiTask models and 

In [ ]:
strategy.experiments

,x,task,y,valid_y,y_pred,y_sd,y_des
0,0.322903,task_B,4.081856e-01,True,NaN,NaN,NaN
1,0.323703,task_B,4.060317e-01,True,NaN,NaN,NaN
2,0.222289,task_A,9.848810e-01,True,NaN,NaN,NaN
3,0.127538,task_A,7.182945e-01,True,NaN,NaN,NaN
4,0.392791,task_A,6.238140e-01,True,1.035736,0.300151,1.035736
5,0.261148,task_A,9.975478e-01,True,0.980886,0.060719,0.980886
6,1.000000,task_B,-4.000000e-01,True,0.490384,0.739088,0.490384
7,0.245150,task_A,9.995357e-01,True,0.998337,0.028762,0.998337
8,0.252561,task_A,9.998706e-01,True,0.999514,0.021829,0.999514
9,0.237606,task_A,9.969692e-01,True,0.996853,0.016992,0.996853
